# 加载预训练模型

In [1]:
import torch
from torchvision import models, transforms

resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

D:\ANACONDA\envs\image-classification\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\ANACONDA\envs\image-classification\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# 预处理图像

In [2]:
import os
from PIL import Image
import numpy as np

In [3]:
def extract_features(image):
    image = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        features = resnet(image)
    return features.squeeze().flatten().numpy()

In [4]:
buildings_features = [extract_features(Image.open('../data/b/' + img_name).convert('RGB')) for img_name in
                      os.listdir('../data/b')]
plants_features = [extract_features(Image.open('../data/p/' + img_name).convert('RGB')) for img_name in
                   os.listdir('../data/p')]
X = np.array(buildings_features + plants_features)
y = np.array([1] * len(buildings_features) + [0] * len(plants_features))

# 拆分数据集，测试集

In [5]:
# 测试集占比20%
# from sklearn.model_selection import train_test_split
# 
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# 用于作业要求的固定10个正样本，10个负样本
# import time


positive_indices = np.where(y == 1)[0]
negative_indices = np.where(y == 0)[0]

# np.random.seed(int(time.time()))
np.random.seed(304)

test_positive_indices = np.random.choice(positive_indices, 10, replace=False)
test_negative_indices = np.random.choice(negative_indices, 10, replace=False)

test_indices = np.concatenate([test_positive_indices, test_negative_indices])
train_indices = np.setdiff1d(np.arange(len(y)), test_indices)

X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

# 初始化及训练

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# 初始化朴素贝叶斯分类器
nb= GaussianNB()

# 训练模型
nb.fit(X_train, y_train)

GaussianNB()

# 测试

In [8]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = nb.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy:  0.85
              precision    recall  f1-score   support

           0       1.00      0.70      0.82        10
           1       0.77      1.00      0.87        10

    accuracy                           0.85        20
   macro avg       0.88      0.85      0.85        20
weighted avg       0.88      0.85      0.85        20

